In [2]:
import json
import pandas as pd
import numpy as np
import time
import requests
import logging
from collections import defaultdict

In [3]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("economic-test")
logger.setLevel(logging.INFO)

In [1]:
def get_information(method, params):
#     url = 'https://api.s0.os.hmny.io/'
    url = 'https://api.s0.t.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [4]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [5]:
def getLastBlockOfCurrentEpoch():
    method = 'hmy_getStakingNetworkInfo'
    params = []
    return get_information(method, params)['result']['epoch-last-block']

In [6]:
def getCurrentAndLastBlock():
    block = getBlockNumber()
    last_block = getLastBlockOfCurrentEpoch()
    print("current and last block numbers", block, last_block)
    return block, last_block

In [4]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [-1]
    return get_information(method, params)['result'] 

In [8]:
def getSuperCommittees():
    method = "hmy_getSuperCommittees"
    params = []
    return get_information(method, params)['result']['current']

In [19]:
def getElectedValidator():
    method = 'hmy_getElectedValidatorAddresses'
    params = []
    return get_information(method, params)['result']

In [9]:
def diffAndFilter(map1, map2):
    map3 = dict()
    for k, v in map2.items():
        if k in map1:
#             if v - map1[k] != 0:
            map3[k] = v - map1[k]
    return map3

In [10]:
# def diffAndFilter2(map1, map2):
#     map3 = dict()
#     for key, val in map2.items():
#         diff = dict()
#         for k, v in map2[key].items():
# #             if v - map1[key][k] != 0:
#             diff[k] = v - map1[key][k]
#         map3[key] = diff
#     return map3

In [35]:
def diffAndFilter2(map1, map2):
    map3 = dict()
    for key, val in map2.items():
        diff = dict()
        for k, v in map2[key].items():
            if key in map1 and k in map1[key]:
                diff[k] = v - map1[key][k]
            if key in map1 and k not in map1[key]:
                diff[k] = map2[key][k]
        map3[key] = diff
    return map3

In [11]:
def getValidatorInfo(validator):
    method = "hmy_getValidatorInformation"
    params = [validator]
    return get_information(method, params)['result']

In [30]:
def R5_test(single):
    logger.info(f"Test-R5: Reward given out to block signers sums up to the total block reward")
    logger.info(f"check sum over earned-reward diff of all keys per shard = 28")
    block, last_block = getCurrentAndLastBlock()
    logger.info(f"current and last block numbers: {block}, {last_block}")
    if block == last_block or block == last_block -1:
        logger.info(f"current at the last block or last second block, wait until the first block in the new epoch")
        while block < last_block+1:
            block = getBlockNumber()
                    
    next_block = block + 1
    while block < next_block:
        block = getBlockNumber()
    time.sleep(3)
    logger.info(f"current block {block}, will begin collecting infos...")

    earn_rewards_prev = defaultdict(dict)
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['metrics']:
            for k in i['metrics']['by-bls-key']:
                key = k['key']
                shard = key['shard-id']
                bls_key = key['bls-public-key']
                earn_reward = k['earned-reward']
                earn_rewards_prev[shard][bls_key] = earn_reward
                
    next_block = block + 1
    while block < next_block:
        block = getBlockNumber()
    time.sleep(3)
    logger.info(f"new block {block} reached, will begin testing...")
    flag = True
    # get the validator info and compute validator rewards
    earn_rewards_curr = defaultdict(dict)
    validator_infos = getAllValidatorInformation()
    block_reward = 28e18
    for i in validator_infos:
        if i['metrics']:
            for k in i['metrics']['by-bls-key']:
                key = k['key']
                shard = key['shard-id']
                bls_key = key['bls-public-key']
                earn_reward = k['earned-reward']
                earn_rewards_curr[shard][bls_key] = earn_reward
    earn_reward_diff = diffAndFilter2(earn_rewards_prev, earn_rewards_curr) 
    for shard, diff in earn_reward_diff.items():
        reward = sum(diff.values())
        if format(reward, '.20e') != format(block_reward, '.20e'):
            logger.warning(f"Test-R5: Fail")
            logger.warning(f"shard: {shard}, block: {block}, sum of earned reward: {reward:.20e}, block reward: {block_reward:.20e}\n")
            flag = False
    
    if single:
        curr_test = None
    else:
        curr_test = R6_test
    if flag:
        logger.info(f"Test-R5: Succeed\n")
        return True, curr_test
    else:
        return False, curr_test

In [31]:
R5_test(True)

INFO:economic-test:Test-R5: Reward given out to block signers sums up to the total block reward
INFO:economic-test:check sum over earned-reward diff of all keys per shard = 28
INFO:economic-test:current and last block numbers: 3230048, 3230119


current and last block numbers 3230048 3230119


INFO:economic-test:current block 3230049, will begin collecting infos...





(False, None)

In [15]:
validator_infos = getAllValidatorInformation()

In [21]:
elected = getElectedValidator()
elected

['one1sgcpjc405ueglhp5udsskjxcn8crrc2lmuf35c',
 'one1qy33jw3g92l40sk03g3kd74hmhszkz23ke8lzs',
 'one1nlptlw8srthgljachm4w5rgv8ulvkt3cgk4uqq',
 'one1uqu9jf8wnxz0j5fe00l67fnl38snvdknxu5lgg',
 'one1lz2mpdnyssmh2x78cf50669hkpd4zx9d0tf2d0',
 'one1km7xg8e3xjys7azp9f4xp8hkw79vm2h3f2lade',
 'one18vn078vyp5jafma8q7kek6w0resrgex9yufqws',
 'one16295hjtqyr0z22swaqthv7mvmvn2gltnj5gera']

In [26]:
def R5_test(single):
    logger.info(f"check sum over earned-reward diff of all keys per shard = validator reward diff")
    block, last_block = getCurrentAndLastBlock()
    logger.info(f"current and last block numbers: {block}, {last_block}")
    if block == last_block or block == last_block -1:
        logger.info(f"current at the last block or last second block, wait until the first block in the new epoch")
        while block < last_block+1:
            block = getBlockNumber()
    logger.info(f"current block {block}, will begin collecting infos...")
    acc_reward_prev = defaultdict(int)
    earn_rewards_prev = defaultdict(dict)
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['metrics']:
            addr = i['validator']['address']
            acc_reward_prev[addr] = i['lifetime']['reward-accumulated']
            for k in i['metrics']['by-bls-key']:
                key = k['key']
                bls_key = key['bls-public-key']
                earn_reward = k['earned-reward']
                earn_rewards_prev[addr][bls_key] = earn_reward
                acc_reward_prev[addr] = i['lifetime']['reward-accumulated']

    next_block = block + 1
    while block < next_block:
        block = getBlockNumber()
    logger.info(f"new block {block} reached, will begin testing...")
    flag = True
    # get the validator info and compute validator rewards
    earn_rewards_curr = defaultdict(dict)
    acc_reward_curr = defaultdict(int)
    validator_infos = getAllValidatorInformation()
    block_reward = 28e18
    for i in validator_infos:
        if i['metrics']:
            addr = i['validator']['address']
            acc_reward_curr[addr] = i['lifetime']['reward-accumulated']
            for k in i['metrics']['by-bls-key']:
                key = k['key']
                bls_key = key['bls-public-key']
                earn_reward = k['earned-reward']
                earn_rewards_curr[addr][bls_key] = earn_reward

    earn_reward_diff = diffAndFilter2(earn_rewards_prev, earn_rewards_curr) 
    val_reward_diff = diffAndFilter(acc_reward_prev, acc_reward_curr)
    for addr, diff in earn_reward_diff.items():
        reward = sum(diff.values())
        val_reward = val_reward_diff[addr]
        if format(reward, '.20e') != format(val_reward, '.20e'):
            logger.warning(f"Test-R5: Fail")
            logger.warning(f"validator: {addr}, sum of earned reward: {reward:.20e}, validator reward: {val_reward:.20e}\n")
            flag = False

    if single:
        curr_test = None
    else:
        curr_test = R6_test
    if flag:
        logger.info(f"Test-R5: Succeed\n")
        return True, curr_test
    else:
        return False, curr_test

In [27]:
R5_test(True)

INFO:economic-test:check sum over earned-reward diff of all keys per shard = validator reward diff
INFO:economic-test:current and last block numbers: 3230024, 3230119
INFO:economic-test:current block 3230024, will begin collecting infos...


current and last block numbers 3230024 3230119


INFO:economic-test:new block 3230025 reached, will begin testing...
INFO:economic-test:Test-R5: Succeed



(True, None)

In [28]:
def R5_test(single):
    logger.info(f"each key earned-reward diff should be equal to voting power * 28")
    block, last_block = getCurrentAndLastBlock()
    logger.info(f"current and last block numbers: {block}, {last_block}")
    if block == last_block or block == last_block -1:
        logger.info(f"current at the last block or last second block, wait until the first block in the new epoch")
        while block < last_block+1:
            block = getBlockNumber()
    next_block = block + 1
    while block < next_block:
        block = getBlockNumber()
    logger.info(f"current block {block}, will begin collecting infos...")
    time.sleep(5)
    earn_rewards_prev = defaultdict(int)
    voting_power = defaultdict(dict)
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['metrics']:
            for k in i['metrics']['by-bls-key']:
                key = k['key']
                bls_key = key['bls-public-key']
                earn_reward = k['earned-reward']
                earn_rewards_prev[bls_key] = earn_reward
                perc = float(key['group-percent'])
                voting_power[bls_key] = perc
                
    next_block = block + 1
    while block < next_block:
        block = getBlockNumber()
    logger.info(f"new block {block} reached, will begin testing...")
    flag = True
    time.sleep(5)
    # get the validator info and compute validator rewards
    earn_rewards_curr = defaultdict(dict)
    
    validator_infos = getAllValidatorInformation()
    block_reward = 28e18
    for i in validator_infos:
        if i['metrics']:
            for k in i['metrics']['by-bls-key']:
                key = k['key']
                bls_key = key['bls-public-key']
                earn_reward = k['earned-reward']
                earn_rewards_curr[bls_key] = earn_reward
                
                
    earn_reward_diff = diffAndFilter(earn_rewards_prev, earn_rewards_curr) 
    for k, v in earn_reward_diff.items():
        reward = voting_power[k] * block_reward
        if format(v, '.12e') != format(reward, '.12e'):
            logger.warning(f"Test-R5: Fail")
            logger.warning(f"bls-key: {k}, earned reward: {v:.12e}, voting-power * 28: {reward:.12e}\n")
            flag = False
  
    if single:
        curr_test = None
    else:
        curr_test = R6_test
    if flag:
        logger.info(f"Test-R5: Succeed\n")
        return True, curr_test
    else:
        return False, curr_test

In [29]:
R5_test(True)

INFO:economic-test:each key earned-reward diff should be equal to voting power * 28
INFO:economic-test:current and last block numbers: 3230031, 3230119


current and last block numbers 3230031 3230119


INFO:economic-test:current block 3230032, will begin collecting infos...
INFO:economic-test:new block 3230033 reached, will begin testing...










(False, None)

In [32]:
elected = getElectedValidator()
elected

['one1sgcpjc405ueglhp5udsskjxcn8crrc2lmuf35c',
 'one1qy33jw3g92l40sk03g3kd74hmhszkz23ke8lzs',
 'one1nlptlw8srthgljachm4w5rgv8ulvkt3cgk4uqq',
 'one1uqu9jf8wnxz0j5fe00l67fnl38snvdknxu5lgg',
 'one1lz2mpdnyssmh2x78cf50669hkpd4zx9d0tf2d0',
 'one1km7xg8e3xjys7azp9f4xp8hkw79vm2h3f2lade',
 'one18vn078vyp5jafma8q7kek6w0resrgex9yufqws',
 'one16295hjtqyr0z22swaqthv7mvmvn2gltnj5gera']

In [36]:
i = elected[0]
iterations = 0
num = 10
block = getBlockNumber()
keys = getValidatorInfo(i)['metrics']['by-bls-key']
prev_reward = keys[0]['earned-reward']
while iterations < num:
    next_block = block + 1
    while block < next_block:
        block = getBlockNumber()
    print(f"block: {block}")
    if getValidatorInfo(i)['metrics']:
        keys = getValidatorInfo(i)['metrics']['by-bls-key']
        for k in keys:
            print(f"group percentage: {k['key']['group-percent']}, perc * 28e18: {float(k['key']['group-percent']) * 28e18}")
            curr_reward = k['earned-reward']
            reward_diff = curr_reward - prev_reward
            print(f"earn reward: {float(reward_diff)}")
        prev_reward = curr_reward
    iterations += 1

block: 3230082
group percentage: 0.243891194098662978, perc * 28e18: 6.828953434762564e+18
earn reward: 0.0
block: 3230083
group percentage: 0.243891194098662978, perc * 28e18: 6.828953434762564e+18
earn reward: 2.882763654056684e+18
block: 3230084
group percentage: 0.243891194098662978, perc * 28e18: 6.828953434762564e+18
earn reward: 2.882763654056684e+18
block: 3230085
group percentage: 0.243891194098662978, perc * 28e18: 6.828953434762564e+18
earn reward: 2.882763654056684e+18
block: 3230086
group percentage: 0.243891194098662978, perc * 28e18: 6.828953434762564e+18
earn reward: 2.882763654056684e+18
block: 3230087
group percentage: 0.243891194098662978, perc * 28e18: 6.828953434762564e+18
earn reward: 2.882763654056684e+18
block: 3230088
group percentage: 0.243891194098662978, perc * 28e18: 6.828953434762564e+18
earn reward: 2.882763654056684e+18
block: 3230089
group percentage: 0.243891194098662978, perc * 28e18: 6.828953434762564e+18
earn reward: 3.2228900916582804e+18
block: 32

In [7]:
getAllValidatorInformation()[0]['validator']['delegations'][0]['amount']/1e18

509495.99999999994